## First, imports:

In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import cm
from scipy.interpolate import interp1d

from astropy import units
from astropy.cosmology import FlatLambdaCDM, z_at_value

Import my library:

In [ ]:
import os
import sys

apt_path = os.path.abspath(os.path.join('..', 'apostletools'))
sys.path.append(apt_path)

import simulation
import simtrace
import match_halo
import dataset_comp
import curve_fit

In [ ]:
import importlib
importlib.reload(simulation)
importlib.reload(simtrace)
importlib.reload(match_halo)
importlib.reload(dataset_comp)
importlib.reload(curve_fit)

# $v_\mathrm{max}$ at Fall-in

To look for a reasonable low-mass limit for satellites subhalo, I inspect the relation of the $v_\mathrm{max}$ values at $z=0$ and at the time, when they fell into orbit of their respective centrals.

In [ ]:
snap_id_ref = 127
snap_id_z0 = 127

---

### LR Simulations

Set the envelope file path, and define the M31 and the MW at redshift $z=0$:

In [ ]:
env_path = os.path.abspath(os.path.join('..', 'test_tracing_inj'))

data = {
    "plain-LCDM-LR": {
        "Simulation": simulation.Simulation("V1_LR_fix", env_path=env_path),
        "Color": ['black', 'gray'],
        "M31_z0": (1, 0),
        "MW_z0": (2, 0)
    },
    "curv-p082-LR": {
        "Simulation": simulation.Simulation("V1_LR_curvaton_p082_fix", env_path=env_path),
        "Color": ['red', 'pink'],
        "M31_z0": (1, 0),
        "MW_z0": (1, 1)
    },
    "curv-p084-LR": {
        "Simulation": simulation.Simulation("V1_LR_curvaton_p084_fix", env_path=env_path),
        "Color": ['blue', 'lightblue'],
        "M31_z0": (1, 0),
        "MW_z0": (1, 0)
    }
}

In [ ]:
env_path = os.path.abspath(os.path.join('..', 'test_tracing_inj'))

data = {
    "plain-LCDM-LR": {
        "Simulation": simulation.Simulation("V1_LR_fix", env_path=env_path),
        "ColorMap": plt.cm.Blues,
        "Color": ['black', 'gray'],
        "M31_z0": (1, 0),
        "MW_z0": (2, 0)
    }
}

---

### Medium Resolution Simulations

Set the envelope file path, and define the M31 and the MW at redshift $z=0$:

In [ ]:
env_path = os.path.abspath(os.path.join('..', 'test_tracing_inj'))

data = {
    "plain-LCDM": {
        "Simulation": simulation.Simulation("V1_MR_fix", env_path=env_path),
        "Color": ['black', 'gray'],
        "Colormap": cm.Blues,
        "M31_z0": (1, 0),
        "MW_z0": (2, 0)
    },
    "curv-p082": {
        "Simulation": simulation.Simulation("V1_MR_curvaton_p082_fix", env_path=env_path),
        "Color": ['red', 'pink'],
        "Colormap": cm.Reds,
        "M31_z0": (1, 0),
        "MW_z0": (1, 1)
    }
}

In [ ]:
env_path = os.path.abspath(os.path.join('..', 'test_tracing_inj'))

data = {
    "plain-LCDM-MR": {
        "Simulation": simulation.Simulation("V1_MR_fix", env_path=env_path),
        "Color": ['black', 'gray'],
        "M31_z0": (1, 0),
        "MW_z0": (2, 0)
    }
}

---

## Tracing

Set the range of snapshots to be traced:

In [ ]:
snap_start = 100
snap_stop = 128
snap_ids = np.arange(snap_start, snap_stop)

Link all subhalos in the simulation, create Subhalo objects for all the individual subhalos found, and write pointers to these objects for each snapshot:

In [ ]:
matcher = match_halo.SnapshotMatcher(n_link_ref=20, n_matches=1)

for sim_data in data.values():
    sim = sim_data["Simulation"]

    # If the simulations are not already linked:
    mtree = simtrace.MergerTree(sim, matcher=matcher, branching="BackwardBranching")
    mtree.build_tree(snap_start, snap_stop)

    # Trace subhalos and get the M31 and the MW Subhalo objects:
    sub_dict = sim.trace_subhalos(snap_start, snap_stop)
    sim_data["Subhalos"] = sub_dict

Get the M31 and the MW halos and compute masking arrays for their satellites (and isolated subhalos) at `snap_id_ref`:

In [ ]:
for sim_data in data.values():
    sim = sim_data["Simulation"]
    sub_dict = sim_data["Subhalos"]
    
    # Get the M31 subhalo:
    m31_id = sim_data["M31_z0"]
    m31 = sub_dict[snap_id_z0][
        sim.get_snapshot(snap_id_z0).index_of_halo(m31_id[0], m31_id[1])
    ]
    sim_data["M31"] = m31 
    
    # Get the MW subhalo:
    mw_id = sim_data["MW_z0"]
    mw = sub_dict[snap_id_z0][
        sim.get_snapshot(snap_id_z0).index_of_halo(mw_id[0], mw_id[1])
    ]
    sim_data["MW"] = mw
    
    # Get masking arrays for satellites (at z=z_ref):
    mask_m31, mask_mw, mask_isol = dataset_comp.split_satellites_by_distance(
        sim.get_snapshot(snap_id_ref), m31_id, mw_id, sat_r=300, isol_r=2000, comov=True
    )
    sim_data["Ref_Selections"] = {"M31_Satellites": mask_m31,
                                  "MW_Satellites": mask_mw,
                                  "LG_Satellites": np.logical_or(mask_m31, mask_mw),
                                  "Isolated": mask_isol}

---

## Retrieve the Datasets

Read all datasets into dictionaries by snapshot:

In [ ]:
# Define the cosmology (should be the same for each simulation):
for sim_data in data.values():
    H0 = sim_data["Simulation"].get_snapshot(snap_stop-1)\
        .get_attribute("HubbleParam", "Header")
    Om0 = sim_data["Simulation"].get_snapshot(snap_stop-1)\
        .get_attribute("Omega0", "Header")
#     print(H0, Om0)
cosmo = FlatLambdaCDM(H0=100 * H0, Om0=Om0) 

In [ ]:
sat_low_lim = 10
isol_low_lim = 15

for sim_data in data.values():
    sim = sim_data["Simulation"]
    sub_dict = sim_data["Subhalos"]
    
    # Get snapshot redshifts and the respective lookback times:
    redshift = sim.get_attribute("Redshift", "Header", snap_ids)
    lookback_time = cosmo.age(0).value - np.array([cosmo.age(z).value for z in redshift])
    sim_data["Redshift"] = redshift
    sim_data["LookbackTime"] = lookback_time
    
    # Get v_max at snap_ref and at the time of fallin:
    vmax_dict = {snap_id: vmax_arr[:, 0] * units.cm.to(units.km) for snap_id, vmax_arr in
                 sim.get_subhalos(snap_ids, "Max_Vcirc", "Extended").items()}
    sim_data["Vmax"] = vmax_dict[snap_id_ref]
    
    fallin_m31, fallin_mw = simtrace.get_fallin_times_lg(
        sim, sim_data["M31"], sim_data["MW"], snap_start, snap_stop, first_infall=True
    )    
    vmax_fallin_m31, snap_id_fallin_m31 = dataset_comp.get_subhalos_at_fallin(
        sub_dict[snap_id_ref], fallin_m31, vmax_dict
    )    
    vmax_fallin_mw, snap_id_fallin_mw = dataset_comp.get_subhalos_at_fallin(
        sub_dict[snap_id_ref], fallin_mw, vmax_dict
    )
    
    sim_data["Vmax_Fallin_M31"] = vmax_fallin_m31
    sim_data["Vmax_Fallin_MW"] = vmax_fallin_mw
    sim_data["Vmax_Fallin"] = np.where(~np.isnan(vmax_fallin_m31), 
                                       vmax_fallin_m31,
                                       vmax_fallin_mw)
                                       
    snap_id_fallin = np.where(~np.isnan(snap_id_fallin_m31),
                              snap_id_fallin_m31,
                              snap_id_fallin_mw)
    inds = np.searchsorted(snap_ids, snap_id_fallin)
    inds[inds == snap_ids.size] = -1
    sim_data["Time_Fallin"] = np.where(inds != -1, lookback_time[inds], np.nan)
    
    # Masking arrays for subhalos at snap_ref:
    snap_ref = sim.get_snapshot(snap_id_ref)
    mask_lum, mask_dark = dataset_comp.split_luminous(snap_ref)
    sim_data["Ref_Selections"].update({
        "Vmax_Sat": dataset_comp.prune_vmax(snap_ref, low_lim=sat_low_lim),
        "Vmax_Isol": dataset_comp.prune_vmax(snap_ref, low_lim=isol_low_lim),
        "Luminous": mask_lum,
        "Dark": mask_dark
    })

In [ ]:
min_time = round(min(list(data.values())[0]['LookbackTime']), 5)
max_time = round(max(list(data.values())[0]['LookbackTime']), 5)
print(min_time, max_time)

norm_func = plt.Normalize(vmin=min_time, vmax=max_time)

In [ ]:
data["plain-LCDM"]["Colormap"] = cm.Greys
data["curv-p082"]["Colormap"] = cm.Reds

In [ ]:
# shades of blue, mapping values from minval to maxval
minval = 0.2
maxval = 1

for sim_data in data.values():
    cmap = sim_data["Colormap"]

    sim_data["Colormap"] = LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, 100))
    )

### Set Plot Parameters

In [ ]:
# Choose font sizes:
parameters = {'axes.titlesize': 10,
              'axes.labelsize': 9,
              'xtick.labelsize': 6,
              'ytick.labelsize': 6,
              'legend.fontsize': 8}

ms = 10 # Marker size
msl = 20
a = 0.75 # Transparency

### Check that colormaps will align

In [ ]:
for sim_data in data.values():
    mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                          sim_data['Ref_Selections']['Dark'])
    fi_dark = sim_data['Time_Fallin'][mask]

    mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                          sim_data['Ref_Selections']['Luminous'])
    fi_lum = sim_data['Time_Fallin'][mask]

    print(min(fi_dark), max(fi_dark))
    print(min(fi_lum), max(fi_lum))

In [ ]:
min_time = 10**10
max_time = 0

for sim_data in data.values():
    mask_dark = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                          sim_data['Ref_Selections']['Dark'])
    mask_lum = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                          sim_data['Ref_Selections']['Luminous'])
    
    min_time = min([min_time, sim_data['Time_Fallin'][mask_dark].min(), 
                    sim_data['Time_Fallin'][mask_lum].min()])
    max_time = max([max_time, sim_data['Time_Fallin'][mask_dark].max(), 
                    sim_data['Time_Fallin'][mask_lum].max()])
min_time = round(min_time, 5)
max_time = round(max_time, 5)
print(min_time, max_time)

norm_func = plt.Normalize(vmin=min_time, vmax=max_time)

In [ ]:
min_time = min(list(data.values())[0]['LookbackTime'])
max_time = max(list(data.values())[0]['LookbackTime'])
min_time = round(min_time, 5)
max_time = round(max_time, 5)
print(min_time, max_time)

norm_func = plt.Normalize(vmin=min_time, vmax=max_time)

### Create Blank Plot

In [ ]:
# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

fig, ax = plt.subplots(figsize=(5, 5))
plt.subplots_adjust(wspace=0.05)

# Set axis:
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_box_aspect(0.9) # Set subfigure box side aspect ratio

ax.set_xlim(10, 110)
# ax.set_ylim(10, 110) 

ax.yaxis.set_ticks_position('both')
ax.set_xlabel("$v_\mathrm{max}(z=0) ~ [\mathrm{km/s}]$")
ax.set_ylabel("$v_\mathrm{max}(z=z_\mathrm{fall-in}) ~/~ v_\mathrm{max}(z=0)$")

# ax.set_ylabel("$N_{[\cdot]}(z) ~/~ N_\mathrm{tot}(z=0)$")
# ax.set_title("Satellite Subhalos")

### ... And Plot

In [ ]:
sim_name, sim_data = list(data.items())[0]

cmap = sim_data['Colormap'] 

# Plot dark
# ---------

mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                      sim_data['Ref_Selections']['Dark'])
time = sim_data['Time_Fallin'][mask]
x = sim_data['Vmax'][mask]
y = sim_data['Vmax_Fallin'][mask] / x
ax.scatter(x, y, s=ms, edgecolor='none', alpha=a, c=cmap(norm_func(time)),
           label="{} non-SF".format(sim_name))
    
# Plot luminous
# -------------

mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                      sim_data['Ref_Selections']['Luminous'])
time = sim_data['Time_Fallin'][mask]
    
x = sim_data['Vmax'][mask]
y = sim_data['Vmax_Fallin'][mask] / x
# Save output for colorbar:
sc = ax.scatter(x, y, s=msl, facecolor='none', alpha=a, edgecolor=cmap(norm_func(time)),          
                label="{} SF".format(sim_name))

# ax.axhline(15, c="gray", linestyle="dotted")    

sc.set_clim(vmin=min_time, vmax=max_time)
sc.set_cmap(cmap)
cax = fig.add_axes([0.92, 0.25, 0.02, 0.5])
cbar = fig.colorbar(sc, cax=cax, orientation='vertical')
cbar.ax.set_ylabel("Lookback Time at Fall-in")

fig

In [ ]:
for sim_name, sim_data in data.items():

    cmap = sim_data['Colormap'] 

    # Plot dark
    # ---------

    mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                          sim_data['Ref_Selections']['Dark'])
    time = sim_data['Time_Fallin'][mask]
    x = sim_data['Vmax'][mask]
    y = sim_data['Vmax_Fallin'][mask] / x
    ax.scatter(x, y, s=ms, edgecolor='none', alpha=a, c=cmap(norm_func(time)),
               label="{} non-SF".format(sim_name))

    # Plot luminous
    # -------------

    mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                          sim_data['Ref_Selections']['Luminous'])
    time = sim_data['Time_Fallin'][mask]

    x = sim_data['Vmax'][mask]
    y = sim_data['Vmax_Fallin'][mask] / x
    # Save output for colorbar:
    sc = ax.scatter(x, y, s=msl, facecolor='none', alpha=a, edgecolor=cmap(norm_func(time)),          
                    label="{} SF".format(sim_name))

    # ax.axhline(15, c="gray", linestyle="dotted")    

sc.set_clim(vmin=min_time, vmax=max_time)
sc.set_cmap(cmap)
cax = fig.add_axes([0.92, 0.25, 0.02, 0.5])
cbar = fig.colorbar(sc, cax=cax, orientation='vertical')
cbar.ax.set_ylabel("Lookback Time at Fall-in")

fig

### Add Median Curves

In [ ]:
n_median_points = 5
min_dx = 0.1
for i, (sim_name, sim_data) in enumerate(data.items()):
    
    mask = sim_data['Ref_Selections']['LG_Satellites']
    time = sim_data['Time_Fallin'][mask]
    x = sim_data['Vmax'][mask]
    y = sim_data['Vmax_Fallin'][mask] / x
    
    x = np.log10(x)
    y = np.log10(y)
    median = curve_fit.median_once_more(
        x, y, n_points_per_bar=n_median_points, min_dx=min_dx
    )
    if median is not None:
        ax.plot(10**median[0], 10**median[1], 
                     sim_data['Color'][0], linestyle='--')
    else:
        print("Could not fit median for:", sim_name)
        
fig

## Plot Fall-in Time

### Create Blank Plot

In [ ]:
min_vmax = 10**10
max_vmax = 0

for sim_data in data.values():
    mask_dark = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                          sim_data['Ref_Selections']['Dark'])
    mask_lum = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                          sim_data['Ref_Selections']['Luminous'])
    
    min_vmax = min([min_vmax, sim_data['Vmax'][mask_dark].min(), 
                    sim_data['Vmax'][mask_lum].min()])
    max_vmax = max([max_vmax, sim_data['Vmax'][mask_dark].max(), 
                    sim_data['Vmax'][mask_lum].max()])

print(min_vmax, max_vmax)
norm_func = plt.Normalize(vmin=min_vmax, vmax=max_vmax)

In [ ]:
# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

fig, ax = plt.subplots(ncols=len(list(data.keys())), figsize=(10, 5), sharey=True)
plt.subplots_adjust(wspace=0.05)

# Set axis:
# ax.set_xscale('log')
ax.set_yscale('log')
ax.set_box_aspect(0.9) # Set subfigure box side aspect ratio

# ax.set_xlim(10, 110)
# ax.set_ylim(10, 110) 

ax.yaxis.set_ticks_position('both')
ax.set_xlabel("Lookback time at fall-in")
ax.set_ylabel("$v_\mathrm{max}(z=z_\mathrm{fall-in}) ~/~ v_\mathrm{max}(z=0)$")

# ax.set_ylabel("$N_{[\cdot]}(z) ~/~ N_\mathrm{tot}(z=0)$")
# ax.set_title("Satellite Subhalos")

### ... And Plot

In [ ]:
sim_name, sim_data = list(data.items())[0]

cmap = sim_data['ColorMap'] 

# Plot dark
# ---------

mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                      sim_data['Ref_Selections']['Dark'])
time = sim_data['Time_Fallin'][mask]
y = sim_data['Vmax_Fallin'][mask] / sim_data['Vmax'][mask]
y = sim_data['Vmax'][mask]
ax.scatter(time, y, s=ms, edgecolor='none', alpha=a, c=cmap(norm_func(sim_data['Vmax'][mask])),
           label="{} non-SF".format(sim_name))
    
# Plot luminous
# -------------

mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                      sim_data['Ref_Selections']['Luminous'])
time = sim_data['Time_Fallin'][mask]
y = sim_data['Vmax_Fallin'][mask] / sim_data['Vmax'][mask]
y = sim_data['Vmax'][mask]
# Save output for colorbar:
sc = ax.scatter(time, y, s=msl, facecolor='none', alpha=a, edgecolor=cmap(norm_func(sim_data['Vmax'][mask])),          
                label="{} SF".format(sim_name))

# ax.axhline(15, c="gray", linestyle="dotted")    

sc.set_cmap(cmap)
cax = fig.add_axes([0.92, 0.25, 0.02, 0.5])
cbar = fig.colorbar(sc, cax=cax, orientation='vertical')
cbar.ax.set_ylabel("Lookback Time at Fall-in")

fig

### Create Blank Plot

In [ ]:
# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

fig, axes = plt.subplots(ncols=len(list(data.keys())), figsize=(10, 5), sharey=True)
plt.subplots_adjust(wspace=0.05)

# Set axis:
axes[0].set_yscale('log')
axes[0].set_ylim(10, 110) 
axes[0].yaxis.set_ticks_position('both')
axes[0].set_ylabel("$v_\mathrm{max}(z=z_\mathrm{fall-in}) ~ [\mathrm{km/s}]$")

for ax, sim_name in zip(axes, list(data.keys())):
    ax.set_xscale('log')
    ax.set_box_aspect(0.9) # Set subfigure box side aspect ratio
    ax.set_xlim(10, 110)
    ax.set_xlabel("$v_\mathrm{max}(z=0) ~ [\mathrm{km/s}]$")
    ax.set_title(sim_name)


# ax.set_ylabel("$N_{[\cdot]}(z) ~/~ N_\mathrm{tot}(z=0)$")
# ax.set_title("Satellite Subhalos")

### ... And Plot

In [ ]:
for i, (sim_name, sim_data) in enumerate(data.items()):

    # Plot luminous
    # -------------
    
    mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                          sim_data['Ref_Selections']['Luminous'])
    print(np.sum(mask))
    axes[i].scatter(sim_data['Vmax'][mask], sim_data['Vmax_Fallin'][mask], 
                    s=msl, edgecolor='none', alpha=a,
                    c=sim_data['Time_Fallin'][mask], cmap=sim_data['Cmap'],
                    label="{} SF".format(sim_name))
    
    # Plot white dots in the centre to indicate luminous
    axes[i].scatter(sim_data['Vmax'][mask], sim_data['Vmax_Fallin'][mask], 
                    s=msl/2, edgecolor='none', alpha=1, c='white')
                
    # Plot dark
    # ---------
    
    mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                          sim_data['Ref_Selections']['Dark'])
    
    # Save output for colorbar:
    sc = axes[i].scatter(sim_data['Vmax'][mask], sim_data['Vmax_Fallin'][mask], 
                s=ms, edgecolor='none', alpha=a,
                c=sim_data['Time_Fallin'][mask], cmap=sim_data['Cmap'],
                label="{} non-SF".format(sim_name))
    
# ax.axhline(15, c="gray", linestyle="dotted")    

cax = fig.add_axes([0.92, 0.25, 0.02, 0.5])
cbar = fig.colorbar(sc, cax=cax, orientation='vertical')
cbar.ax.set_ylabel("Lookback Time at Fall-in")

fig

### Create Blank Plot

In [ ]:
# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

fig, axes = plt.subplots(ncols=len(list(data.keys())), figsize=(10, 5), sharey=True)
plt.subplots_adjust(wspace=0.05)

# Set axis:
# axes[0].set_yscale('log')
# axes[0].set_ylim(10, 110) 
axes[0].yaxis.set_ticks_position('both')
axes[0].set_ylabel("$v_\mathrm{max}(z=z_\mathrm{fall-in}) ~ [\mathrm{km/s}]$")

for ax, sim_name in zip(axes, list(data.keys())):
    ax.set_xscale('log')
    ax.set_box_aspect(0.9) # Set subfigure box side aspect ratio
    ax.set_xlim(10, 110)
    ax.set_xlabel("$v_\mathrm{max}(z=0) ~ [\mathrm{km/s}]$")
    ax.set_title(sim_name)


# ax.set_ylabel("$N_{[\cdot]}(z) ~/~ N_\mathrm{tot}(z=0)$")
# ax.set_title("Satellite Subhalos")

### ... And Plot

In [ ]:
for i, (sim_name, sim_data) in enumerate(data.items()):

    # Plot luminous
    # -------------
    
    mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                          sim_data['Ref_Selections']['Luminous'])
    print(np.sum(mask))
    axes[0].scatter(sim_data['Vmax'][mask], sim_data['Vmax_Fallin'][mask] / sim_data['Vmax'][mask], 
                    s=msl, edgecolor='none', alpha=a,
                    c=sim_data['Time_Fallin'][mask], cmap=sim_data['Cmap'],
                    label="{} SF".format(sim_name))
    
    # Plot white dots in the centre to indicate luminous
    axes[0].scatter(sim_data['Vmax'][mask], sim_data['Vmax_Fallin'][mask] / sim_data['Vmax'][mask], 
                    s=msl/2, edgecolor='none', alpha=1, c='white')
                
    # Plot dark
    # ---------
    
    mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                          sim_data['Ref_Selections']['Dark'])
    
    # Save output for colorbar:
    sc = axes[0].scatter(sim_data['Vmax'][mask], sim_data['Vmax_Fallin'][mask] / sim_data['Vmax'][mask], 
                s=ms, edgecolor='none', alpha=a,
                c=sim_data['Time_Fallin'][mask], cmap=sim_data['Cmap'],
                label="{} non-SF".format(sim_name))
    
# ax.axhline(15, c="gray", linestyle="dotted")    

cax = fig.add_axes([0.92, 0.25, 0.02, 0.5])
cbar = fig.colorbar(sc, cax=cax, orientation='vertical')
cbar.ax.set_ylabel("Lookback Time at Fall-in")

fig

In [ ]:
for i, (sim_name, sim_data) in enumerate(data.items()):

    # Plot luminous
    # -------------
    
    mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                          sim_data['Ref_Selections']['Luminous'])
    
#     mask = np.logical_and(mask, sim_data['Ref_Selections']['Vmax_Sat'])
    
    axes[i].scatter(sim_data['Vmax'][mask], sim_data['Vmax_Fallin'][mask], 
                    s=msl, edgecolor='none', alpha=a,
                    c=sim_data['Time_Fallin'][mask], cmap='Blues',
                    label="{} SF".format(sim_name))
    
    # Plot white dots in the centre to indicate luminous
    axes[i].scatter(sim_data['Vmax'][mask], sim_data['Vmax_Fallin'][mask], 
                    s=msl/2, edgecolor='none', alpha=1, c='white')
                
    # Plot dark
    # ---------
    
    mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                          sim_data['Ref_Selections']['Dark'])
    
#     mask = np.logical_and(mask, sim_data['Ref_Selections']['Vmax_Sat'])
    
    # Save output for colorbar:
    sc = axes[i].scatter(sim_data['Vmax'][mask], sim_data['Vmax_Fallin'][mask], 
                s=ms, edgecolor='none', alpha=a,
                c=sim_data['Time_Fallin'][mask], cmap='Blues',
                label="{} non-SF".format(sim_name))
    
# ax.axhline(15, c="gray", linestyle="dotted")    

cax = fig.add_axes([0.92, 0.25, 0.02, 0.5])
cbar = fig.colorbar(sc, cax=cax, orientation='vertical')
cbar.ax.set_ylabel("Lookback Time at Fall-in")

fig

### Save the Figures

In [ ]:
filename = 'vmax_at_fallin'
for name in data.keys():
    filename += '_{}'.format(name)
filename += '.png'
    
path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))
filename = os.path.join(path, filename)

fig.savefig(filename, dpi=300, bbox_inches='tight')

### Alternate attempts at plotting

In [ ]:
# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

fig, ax = plt.subplots(nrows=len(list(data.keys())), figsize=(5, 5), sharey=True)
plt.subplots_adjust(wspace=0.05)

ms = 15 # Marker size
msl = 20
a = 0.9 # Transparency


# Set axis:
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_box_aspect(0.9) # Set subfigure box side aspect ratio

ax.set_xlim(10, 110)
ax.set_ylim(10, 110) 

ax.yaxis.set_ticks_position('both')
ax.set_xlabel("$v_\mathrm{max}(z=0) ~ [\mathrm{km/s}]$")
ax.set_ylabel("$v_\mathrm{max}(z=z_\mathrm{fall-in}) ~ [\mathrm{km/s}]$")

# ax.set_ylabel("$N_{[\cdot]}(z) ~/~ N_\mathrm{tot}(z=0)$")
# ax.set_title("Satellite Subhalos")

### ... And Plot

In [ ]:
# Plot dark:
toolight = int(snap_ids.size / 4)
snap_colors = plt.cm.Blues(np.linspace(0, 1, snap_ids.size + toolight))[toolight:]
#     colors = plt.cm.viridis(np.linspace(0, 1, snap_ids.size))

mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                      sim_data['Ref_Selections']['Dark'])
x = sim_data['Vmax'][mask]
y = sim_data['Vmax_Fallin'][mask]

# colors = plt.cm.Blues(sim_data['Time_Fallin'][mask])

colors = sim_data['Time_Fallin'][mask]




sc = ax.scatter(x, y, c=colors, cmap='Blues')

cbar = fig.colorbar(sc, cax=cax, orientation='vertical')
cbar.ax.set_ylabel("Lookback Time at Fall-in")
# sc = ax.scatter(x, y, c=colors)


fig

In [ ]:
for sim_name, sim_data in data.items():
    
    # Plot Satellites
    # ---------------
            
    # Plot dark:
    toolight = int(snap_ids.size / 4)
#     snap_colors = plt.cm.Blues(np.linspace(0, 1, snap_ids.size + toolight))[toolight:]
#     colors = plt.cm.viridis(np.linspace(0, 1, snap_ids.size))
    
    mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                          sim_data['Ref_Selections']['Dark'])
    colors = plt.cm.Blues(sim_data['Time_Fallin'][mask])
    sc = ax.scatter(sim_data['Vmax'][mask], sim_data['Vmax_Fallin'][mask], 
                s=ms, edgecolor='none', alpha=a,
                c=colors,
                label="{} non-SF".format(sim_name))
    
    # Plot dark:
    mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                          sim_data['Ref_Selections']['Luminous'])
    colors = plt.cm.Blues(sim_data['Time_Fallin'][mask])
    ax.scatter(sim_data['Vmax'][mask], sim_data['Vmax_Fallin'][mask], 
                    s=msl, facecolor='none', alpha=a,
                    edgecolors=colors,
                    label="{} SF".format(sim_name))
    
    ax.axhline(15, c="gray", linestyle="dotted")
    
# fig.colorbar(sc, ax=ax)

    # Add colorbar, make sure to specify tick locations to match desired ticklabels
#     cbar = fig.colorbar(cax, ticks=[-1, 0, 1])
#     cbar.ax.set_yticklabels(['< -1', '0', '> 1'])# vertically oriented colorbar
    
#     axes[0].axhline(15, c="gray", linestyle="dotted")
    
#     mask = np.logical_and(sim_data['Ref_Selections']['Isolated'],
#                           sim_data['Ref_Selections']['Dark'])
#     axes[1].scatter(sim_data['Vmax'][mask], sim_data['Rmax'][mask], 
#                     s=ms, edgecolor='none', alpha=a,
#                     c=sim_data['PlotStyle']['Color'][1],
#                     label="{} non-SF".format(sim_name))
    
    
    
#     # Plot all:
#     mask = sim_data["Ref_Selections"]["LG_Satellites"]
#     sat_cnt = count_subhalos(form_snap[mask])
#     sat_num = sat_cnt[-1]
#     axes[0].plot(time, sat_cnt/sat_num, c=sim_data["Color"][1], 
#                  linestyle="dotted")
    
#     # Plot those above the satellite mass limit:
#     mask = np.logical_and(sim_data["Ref_Selections"]["LG_Satellites"],
#                           sim_data["Ref_Selections"]["Vmax_Sat"])
#     axes[0].plot(time, count_subhalos(form_snap[mask])/sat_num, c=sim_data["Color"][1])
    
#     # ...out of those, plot luminous:
#     mask = np.logical_and.reduce([sim_data["Ref_Selections"]["LG_Satellites"],
#                                   sim_data["Ref_Selections"]["Vmax_Sat"],
#                                   sim_data["Ref_Selections"]["Luminous"]])
#     axes[0].plot(time, count_subhalos(form_snap[mask])/sat_num, c=sim_data["Color"][0])

    
#     # Plot Isolated
#     # -------------
    
#     # Plot all:
#     mask = sim_data["Ref_Selections"]["Isolated"]
#     sat_cnt = count_subhalos(form_snap[mask], absolute=True)
#     sat_num = sat_cnt[-1]
#     axes[1].plot(time, sat_cnt/sat_num, c=sim_data["Color"][1], linestyle='dotted')
    
#     # Plot those above the satellite mass limit:
#     mask = np.logical_and(sim_data["Ref_Selections"]["Isolated"],
#                           sim_data["Ref_Selections"]["Vmax_Isol"])
#     axes[1].plot(time, count_subhalos(form_snap[mask])/sat_num, c=sim_data["Color"][1])
    
#     # ...out of those, plot luminous:
#     mask = np.logical_and.reduce([sim_data["Ref_Selections"]["Isolated"],
#                                   sim_data["Ref_Selections"]["Vmax_Isol"],
#                                   sim_data["Ref_Selections"]["Luminous"]])
#     axes[1].plot(time, count_subhalos(form_snap[mask])/sat_num, c=sim_data["Color"][0])

fig

In [ ]:
# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

fig, ax = plt.subplots(nrows=len(list(data.keys())), figsize=(5, 5), sharey=True)
plt.subplots_adjust(wspace=0.05)

ms = 15 # Marker size
msl = 20
a = 0.9 # Transparency


# Set axis:
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_box_aspect(0.9) # Set subfigure box side aspect ratio

ax.set_xlim(10, 110)
ax.set_ylim(10, 110) 

ax.yaxis.set_ticks_position('both')
ax.set_xlabel("$v_\mathrm{max}(z=0) ~ [\mathrm{km/s}]$")
ax.set_ylabel("$v_\mathrm{max}(z=z_\mathrm{fall-in}) ~ [\mathrm{km/s}]$")

# ax.set_ylabel("$N_{[\cdot]}(z) ~/~ N_\mathrm{tot}(z=0)$")
# ax.set_title("Satellite Subhalos")

In [ ]:
for i, (sim_name, sim_data) in enumerate(data.items()):
            
    # Plot dark:  
    mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                          sim_data['Ref_Selections']['Dark'])
    
    sc = ax.scatter(sim_data['Vmax'][mask], sim_data['Vmax_Fallin'][mask], 
                s=ms, edgecolor='none', alpha=a,
                c=sim_data['Time_Fallin'][mask], cmap='Blues',
                label="{} non-SF".format(sim_name))
    
    # Plot luminous:
    mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                          sim_data['Ref_Selections']['Luminous'])
    colors = plt.cm.Blues(sim_data['Time_Fallin'][mask])
    ax.scatter(sim_data['Vmax'][mask], sim_data['Vmax_Fallin'][mask], 
                    s=msl, facecolor='none', alpha=a,
                    edgecolors=colors,
                    label="{} SF".format(sim_name))
    
    ax.axhline(15, c="gray", linestyle="dotted")
    

cax = fig.add_axes([0.76, 0.2, 0.04, 0.5])
cbar = fig.colorbar(sc, cax=cax, orientation='vertical')
cbar.ax.set_ylabel("Lookback Time at Fall-in")


fig

In [ ]:
for i, (sim_name, sim_data) in enumerate(data.items()):
            
    # Plot dark:  
    mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                          sim_data['Ref_Selections']['Dark'])
    
    sc = ax.scatter(sim_data['Vmax'][mask], sim_data['Vmax_Fallin'][mask], 
                s=ms, edgecolor='none', alpha=a,
                c=sim_data['Time_Fallin'][mask], cmap='Blues',
                label="{} non-SF".format(sim_name))
    
    # Plot luminous:
    mask = np.logical_and(sim_data['Ref_Selections']['LG_Satellites'],
                          sim_data['Ref_Selections']['Luminous'])
    colors = plt.get_cmap('Blues')(sim_data['Time_Fallin'][mask])
    ax.scatter(sim_data['Vmax'][mask], sim_data['Vmax_Fallin'][mask], 
                    s=msl, facecolor='none', alpha=a,
                    edgecolors=colors,
                    label="{} SF".format(sim_name))
    
    ax.axhline(15, c="gray", linestyle="dotted")
    

cax = fig.add_axes([0.76, 0.2, 0.04, 0.5])
cbar = fig.colorbar(sc, cax=cax, orientation='vertical')
cbar.ax.set_ylabel("Lookback Time at Fall-in")


fig

In [ ]:
# CRAP !!!!
    
# fig.colorbar(sc, ax=ax)

    # Add colorbar, make sure to specify tick locations to match desired ticklabels
#     cbar = fig.colorbar(cax, ticks=[-1, 0, 1])
#     cbar.ax.set_yticklabels(['< -1', '0', '> 1'])# vertically oriented colorbar
    
#     axes[0].axhline(15, c="gray", linestyle="dotted")
    
#     mask = np.logical_and(sim_data['Ref_Selections']['Isolated'],
#                           sim_data['Ref_Selections']['Dark'])
#     axes[1].scatter(sim_data['Vmax'][mask], sim_data['Rmax'][mask], 
#                     s=ms, edgecolor='none', alpha=a,
#                     c=sim_data['PlotStyle']['Color'][1],
#                     label="{} non-SF".format(sim_name))
    
    
    
#     # Plot all:
#     mask = sim_data["Ref_Selections"]["LG_Satellites"]
#     sat_cnt = count_subhalos(form_snap[mask])
#     sat_num = sat_cnt[-1]
#     axes[0].plot(time, sat_cnt/sat_num, c=sim_data["Color"][1], 
#                  linestyle="dotted")
    
#     # Plot those above the satellite mass limit:
#     mask = np.logical_and(sim_data["Ref_Selections"]["LG_Satellites"],
#                           sim_data["Ref_Selections"]["Vmax_Sat"])
#     axes[0].plot(time, count_subhalos(form_snap[mask])/sat_num, c=sim_data["Color"][1])
    
#     # ...out of those, plot luminous:
#     mask = np.logical_and.reduce([sim_data["Ref_Selections"]["LG_Satellites"],
#                                   sim_data["Ref_Selections"]["Vmax_Sat"],
#                                   sim_data["Ref_Selections"]["Luminous"]])
#     axes[0].plot(time, count_subhalos(form_snap[mask])/sat_num, c=sim_data["Color"][0])

    
#     # Plot Isolated
#     # -------------
    
#     # Plot all:
#     mask = sim_data["Ref_Selections"]["Isolated"]
#     sat_cnt = count_subhalos(form_snap[mask], absolute=True)
#     sat_num = sat_cnt[-1]
#     axes[1].plot(time, sat_cnt/sat_num, c=sim_data["Color"][1], linestyle='dotted')
    
#     # Plot those above the satellite mass limit:
#     mask = np.logical_and(sim_data["Ref_Selections"]["Isolated"],
#                           sim_data["Ref_Selections"]["Vmax_Isol"])
#     axes[1].plot(time, count_subhalos(form_snap[mask])/sat_num, c=sim_data["Color"][1])
    
#     # ...out of those, plot luminous:
#     mask = np.logical_and.reduce([sim_data["Ref_Selections"]["Isolated"],
#                                   sim_data["Ref_Selections"]["Vmax_Isol"],
#                                   sim_data["Ref_Selections"]["Luminous"]])
#     axes[1].plot(time, count_subhalos(form_snap[mask])/sat_num, c=sim_data["Color"][0])

In [ ]:
for sim_data in data.values():
    sim = sim_data["Simulation"]
    
    # Get snapshot redshifts and the respective lookback times:
    redshift = sim.get_attribute("Redshift", "Header", snap_ids)
    lookback_time = cosmo.age(0).value - np.array([cosmo.age(z).value for z in redshift])
    sim_data["Redshift"] = {sid: z for sid, z in zip(snap_ids, redshift)}
    sim_data["LookbackTime"] =  {sid: t for sid, t in zip(snap_ids, lookback_time)}

    # Get the datasets in a dictionary, with items for each snapshot data:
    sim_data["Mass"] = {sid: m * units.g.to(units.Msun) for sid, m in
            sim.get_subhalos(snap_ids, "Mass").items()}
    sim_data["Vmax"] = {sid: vm[:, 0] * units.cm.to(units.km) for sid, vm in
            sim.get_subhalos(snap_ids, "Max_Vcirc", h5_group="Extended").items()}
    sim_data["CentreOfPotential"] = {sid: c * units.cm.to(units.kpc) for sid, c in
           sim.get_subhalos(snap_ids, "CentreOfPotential").items()}

    sim_data["R_M31"] = {sid: np.linalg.norm(d, axis=1) * units.cm.to(units.kpc)
                for sid, d in sim_data["M31"].distance_to_self(snap_ids).items()}
    sim_data["R_MW"] = {sid: np.linalg.norm(d, axis=1) * units.cm.to(units.kpc)
               for sid, d in sim_data["MW"].distance_to_self(snap_ids).items()}

In [ ]:
for sim_data in data.values():
    sim = sim_data["Simulation"]
    sub_dict = sim_data["Subhalos"]

#     sim_data["SurvivalTime"] = {sid: np.array([len(sub.indices) for sub in subs])
#                            for sid, subs in sub_dict.items()}

    print(sim.sim_id)
    print(sim_data["M31"].get_indices())
    print(sim_data["MW"].get_indices())
    fallin_m31, fallin_mw = simtrace_redo.get_fallin_times_lg(
        sim, sim_data["M31"], sim_data["MW"], snap_start, snap_stop
    )
    
    print(fallin_m31[snap_z0].size, sim.get_snapshot(snap_z0).get_subhalo_number())
    print(fallin_m31[snap_z0][:150])
    print(sim_data["Ref_Selections"]["M31_Satellites"][:150])
    
#     vmax_fallin_m31 = dataset_comp.get_subhalos_at_fallin(
#         sub_dict[snap_ref], fallin_m31, sim_data["Vmax"][snap_ref]
#     )

Define masking arrays to select satellites of M31 and MW and random sample of isolated galaxies:

In [ ]:
def random_mask(mask, n):
    """ From the selection prescribed by ´mask´, select ´n´ items at random. """
    k = np.sum(mask)
    mask_rand = np.full(k, False)
    mask_rand[:min(n, k)] = True
    np.random.shuffle(mask_rand)

    mask_new = np.full(mask.size, False)
    mask_new[mask] = mask_rand
    
    return mask_new

In [ ]:
# Get masking arrays for satellites (at z=0):
mask_m31, mask_mw, mask_isol = dataset_comp.split_satellites_by_distance(
    sim.get_snapshot(snap_z0), m31, mw, sat_r=300
)

mask_rand_isol = random_mask(mask_isol, 30)

# # Randomly select ´n_isol´ isolated galaxies:
# n_isol = 30
# mask_rand = np.full(np.sum(mask_isol), False)
# mask_rand[:n_isol] = True
# np.random.shuffle(mask_rand)

# mask_rand_isol = np.full(mask_isol.size, False)
# mask_rand_isol[mask_isol] = mask_rand

Make the selections and write the dataset that are ready for plotting to a dictionary:

In [ ]:
# From the full datasets, read M31 satellite data and add to the data dictionary:
data = {}

m31_sats = sub_dict[snap_z0][mask_m31]
data["M31_Satellites"] = {
    "Snap_id": np.array([np.array(sat.get_indices())[1] for sat in m31_sats]),
    "Redshift": np.array([np.array([redshift[sid] for _, sid in zip(*sat.get_indices())])
                 for sat in m31_sats]),
    "Mass": np.array([dataset_comp.subhalo_dataset_from_dict(sat, mass)[0] for sat in m31_sats]),
    "Vmax": np.array([dataset_comp.subhalo_dataset_from_dict(sat, vmax)[0] for sat in m31_sats]),
    "Distance": np.array([dataset_comp.subhalo_dataset_from_dict(sat, m31_dist)[0] for sat in m31_sats])
}

# ...Same for MW:
mw_sats = sub_dict[snap_z0][mask_mw]
data["MW_Satellites"] = {
    "Snap_id": np.array([np.array(sat.get_indices())[1] for sat in mw_sats]),
    "Redshift": np.array([np.array([redshift[sid] for _, sid in zip(*sat.get_indices())])
                 for sat in mw_sats]),
    "Mass": np.array([dataset_comp.subhalo_dataset_from_dict(sat, mass)[0] for sat in mw_sats]),
    "Vmax": np.array([dataset_comp.subhalo_dataset_from_dict(sat, vmax)[0] for sat in mw_sats]),
    "Distance": np.array([dataset_comp.subhalo_dataset_from_dict(sat, mw_dist)[0] for sat in mw_sats])
}

# ...Same for the randomly selected isolated galaxies:
isol_subs = sub_dict[snap_z0][mask_rand_isol]
data["Isolated"] = {
    "Snap_id": np.array([np.array(sat.get_indices())[1] for sat in isol_subs]),
    "Redshift": np.array([np.array([redshift[sid] for _, sid in zip(*sat.get_indices())])
                 for sat in isol_subs]),
    "Mass": np.array([dataset_comp.subhalo_dataset_from_dict(sat, mass)[0] for sat in isol_subs]),
    "Vmax": np.array([dataset_comp.subhalo_dataset_from_dict(sat, vmax)[0] for sat in isol_subs]),
    "MW_Distance": np.array([dataset_comp.subhalo_dataset_from_dict(sat, mw_dist)[0] for sat in isol_subs])
}

## Subhalo Survival Times

In [ ]:
surv_time = {sid: np.array([len(sub.indices) for sub in subs])
             for sid, subs in sub_dict.items()}

Plot the number of snapshots, through which each snapshot is traced, against its $v_\mathrm{max}$. Below plot the counts of subhalos for each survival time. 

We see that, by far, most subhalos survive through the entire time range, through which we have done the linking. Only a small fraction is traced for less than 5 snapshots.

In [ ]:
fig, axes = plt.subplots(nrows=2)

st = surv_time[snap_z0]
st_unique, st_cnt = np.unique(st, return_counts=True)
v = vmax[snap_z0]

axes[0].scatter(st, v)
axes[1].plot(st_unique, st_cnt)

Let us look at the masses of the subhalos that survive longest vs. those that die shortly, more closely. I divide the subhalos into those that survive the whole linking period, those that survive through less than 3 snapshots, and all in between:

In [ ]:
def select_by_survival_time(surv_times, sel_times):
    """ Compute masking arrays for subhalos which are present through the given 
    ranges of snapshots. 
    
    Parameters
    ----------
    sel_times : list of int
        The edges of the selection ranges. The lower limit is exclusive, and 
        the upper limit is inclusive.
    """
    masking = {}
    for sid, st in surv_times.items():
        masking[sid] = [np.logical_and(st > sel_times[i], st <= sel_times[i + 1])
                        for i in range(len(sel_times) - 1)]
        
    return masking

In [ ]:
masks = select_by_survival_time(surv_time, [0, 5, max(st)-1, max(surv_time[snap_z0])])
masks = masks[snap_z0]

mask_prune = (v < 60)
print(np.sum(np.logical_and(masks[2], mask_prune)))
print(np.sum(np.logical_and(masks[1], mask_prune)))
print(np.sum(np.logical_and(masks[0], mask_prune)))

In [ ]:
mask_prune = (v < 60)
mask_long_surv = np.logical_and((st == max(st)), mask_prune)
print(np.sum(mask_long_surv))
mask_inter_surv = np.logical_and(np.logical_and((st > 5), (st < max(st))), mask_prune)
print(np.sum(mask_inter_surv))
mask_short_surv = np.logical_and((st <= 5), mask_prune)
print(np.sum(mask_short_surv))

Below are histograms for each of these categories (by  $v_\mathrm{max}$).

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
ax.set_xlim(10, 60)

bin_edges = np.linspace(10, 60, 21)
print(bin_edges)

hist_bars = [v[mask_short_surv], v[mask_inter_surv], v[mask_long_surv]]
a = ax.hist(hist_bars, bins=bin_edges, density=True)

print(a)

## Plot M31 Satellites

In [ ]:
def random_mask(mask, n):
    """ From the selection prescribed by ´mask´, select ´n´ items at random. """
    k = np.sum(mask)
    mask_rand = np.full(k, False)
    mask_rand[:min(n, k)] = True
    np.random.shuffle(mask_rand)

    mask_new = np.full(mask.size, False)
    mask_new[mask] = mask_rand
    
    return mask_new

In [ ]:
# Select ´n´ random satellites for plotting:
n=15
sat_arrs = data["M31_Satellites"]["Redshift"]
mask_nonvol = (np.array([arr.size > 1 for arr in sat_arrs]))

mask_sel_m31 = random_mask(mask_nonvol, n)

In [ ]:
fig, ax = plt.subplots()

ax.invert_xaxis()

for r, z in zip(data["M31_Satellites"]["Distance"][mask_sel_m31],
                data["M31_Satellites"]["Redshift"][mask_sel_m31]):
    ax.plot(z, r)
    ax.scatter(z, r, s=10)

In [ ]:
fig, ax = plt.subplots()

ax.invert_xaxis()

for r, z in zip(data["M31_Satellites"]["Distance"][mask_sel_m31],
                data["M31_Satellites"]["Redshift"][mask_sel_m31]):
    f = interp1d(z, r, kind='cubic')
    z_new = np.linspace(min(z), max(z), num=1000)
    ax.plot(z_new, f(z_new))

Add vertical line for the subhalo mass limit: 20 * m_dm-particle

In [ ]:
fig, ax = plt.subplots()

ax.invert_xaxis()
ax.set_yscale('log')

for m, z in zip(data["M31_Satellites"]["Mass"][mask_sel_m31],
                data["M31_Satellites"]["Redshift"][mask_sel_m31]):
    ax.plot(z, m)
#     ax.scatter(z, m, s=10)

In [ ]:
fig, ax = plt.subplots()

ax.invert_xaxis()
ax.set_yscale('log')

for vm, z in zip(data["M31_Satellites"]["Vmax"][mask_sel_m31],
                data["M31_Satellites"]["Redshift"][mask_sel_m31]):
    ax.plot(z, vm)
#     ax.scatter(z, vm, s=10)

In [ ]:
fig, ax = plt.subplots()

ax.invert_xaxis()
ax.set_yscale('log')

n=40
for vm, z in zip(data["M31_Satellites"]["Vmax"][:n],
                 data["M31_Satellites"]["Redshift"][:n]):
    ax.plot(z, vm)
    ax.scatter(z, vm, s=10)

## Plot MW Satellites

In [ ]:
fig, ax = plt.subplots()

ax.invert_xaxis()

n=10
for r, z in zip(data["MW_Satellites"]["Distance"][:n],
                data["MW_Satellites"]["Redshift"][:n]):
    ax.plot(z, r)
    ax.scatter(z, r, s=10)

In [ ]:
fig, ax = plt.subplots()

ax.axvline(sim.get_attribute("Redshift", "Header", snap_z0)[0], c='gray',  linestyle='dotted')

ax.invert_xaxis()

n=20
for r, z in zip(data["MW_Satellites"]["Distance"][:n],
                data["MW_Satellites"]["Redshift"][:n]):
    if r.size > 1:
        f = interp1d(z, r, kind='cubic')
        z_new = np.linspace(min(z), max(z), num=1000)
        ax.plot(z_new, f(z_new))
    #     ax.scatter(z, r, s=10)

In [ ]:
fig, ax = plt.subplots()

ax.axvline(sim.get_attribute("Redshift", "Header", snap_z0)[0], c='gray',  linestyle='dotted')

ax.invert_xaxis()

n=20
for r, z in zip(data["MW_Satellites"]["Distance"],
                data["MW_Satellites"]["Redshift"]):
    if r.size > 1:
        f = interp1d(z, r, kind='cubic')
        z_new = np.linspace(min(z), max(z), num=1000)
        ax.plot(z_new, f(z_new))
    #     ax.scatter(z, r, s=10)

In [ ]:
fig, ax = plt.subplots()

ax.invert_xaxis()
ax.set_yscale('log')

n=40
for vm, z in zip(data["MW_Satellites"]["Vmax"][:n],
                 data["MW_Satellites"]["Redshift"][:n]):
    ax.plot(z, vm)
    ax.scatter(z, vm, s=10)

In [ ]:
fig, ax = plt.subplots()

ax.invert_xaxis()
ax.set_yscale('log')

n=40
for vm, z in zip(data["MW_Satellites"]["Mass"][:n],
                 data["MW_Satellites"]["Redshift"][:n]):
    ax.plot(z, vm)
    ax.scatter(z, vm, s=10)

## Plot random Isolated Subhalos

First, show, which sample:

In [ ]:
gns = sim.get_snapshot(snap_z0).get_subhalos('GroupNumber')
sgns = sim.get_snapshot(snap_z0).get_subhalos('SubGroupNumber')

ids = [(gn, sgn) for gn, sgn in zip(gns[mask_rand_isol], sgns[mask_rand_isol])]
print(ids)

In [ ]:
fig, ax = plt.subplots()

ax.invert_xaxis()

for r, z in zip(data["Isolated"]["MW_Distance"],
                data["Isolated"]["Redshift"]):
    ax.plot(z, r)

In [ ]:
fig, ax = plt.subplots()

ax.invert_xaxis()
ax.set_yscale('log')


for m, z in zip(data["Isolated"]["Mass"],
                data["Isolated"]["Redshift"]):
    ax.plot(z, m)
    ax.scatter(z, m, s=10)

## Further Ideas

With __eq__ and __hash__ implemented, it is simple to read satellites at any time:
- Create a set ´all_satellites´, or similar
- Iterate through snapshots
- At each snapshot, add all satellites to the set

Note that this only works properly, if all the subhalos have been traced as far as possible, since __eq__ and __hash__ are based on the subhalo data at the formation snapshot. 

---

## Crap below

In [ ]:
snap_z0 = 127
for m31_id, mw_id, sim_data in zip(m31_id_z0, mw_id_z0, data.values()):
    sim = sim_data["Simulation"]

    # Trace subhalos and add the M31 and the MW Subhalo objects to the
    # ´sim_data´ dictionary, as well as their satellites:
    sub_dict = simtrace_redo.trace_subhalos(
        sim, snap_start, snap_stop
    )
    
    m31 = sub_dict[snap_z0][
        sim.get_snapshot(snap_z0).index_of_halo(m31_id[0], m31_id[1])
    ]
    mw = sub_dict[snap_z0][
        sim.get_snapshot(snap_z0).index_of_halo(mw_id[0], mw_id[1])
    ]    
    
    m31_dist = m31.distance_to_self(snap_ids)
    mw_dist = mw.distance_to_self(snap_ids)

In [ ]:
snap_z0 = 127
for m31_id, mw_id, sim_data in zip(m31_id_z0, mw_id_z0, data.values()):
    sim = sim_data["Simulation"]
    
    # Get masking arrays for satellites:
    mask_sats,_ = dataset_comp.split_satellites_by_distance(
        sim.get_snapshot(snap_z0), m31_id, mw_id
    )
    mask_m31, mask_mw = mask_sats
    
    # Trace subhalos and add the M31 and the MW Subhalo objects to the
    # ´sim_data´ dictionary, as well as their satellites:
    sub_dict = simtrace_redo.trace_subhalos(
        sim, snap_start, snap_stop
    )
    
    m31 = sub_dict[snap_z0][
        sim.get_snapshot(snap_z0).index_of_halo(m31_id[0], m31_id[1])
    ]
    mw = sub_dict[snap_z0][
        sim.get_snapshot(snap_z0).index_of_halo(mw_id[0], mw_id[1])
    ]


    m31_sats = sub_dict[snap_z0][mask_m31]
    sim_data["M31_Satellites"] = {
        "Snap_id": [np.array([sid for _, sid zip(*sat.get_indices())])
                     for sat in m31_sats],
        "Mass": [dataset_comp.subhalo_dataset_from_dict(sat, mass) for sat in m31_sats],
        "Vmax": [dataset_comp.subhalo_dataset_from_dict(sat, vmax) for sat in m31_sats]
    }

In [ ]:
snap_z0 = 127
for m31_id, mw_id, sim_data in zip(m31_id_z0, mw_id_z0, data.values()):
    sim = sim_data["Simulation"]
    
    # Get masking arrays for satellites:
    mask_sats,_ = dataset_comp.split_satellites_by_distance(
        sim.get_snapshot(snap_z0), m31_id, mw_id
    )
    mask_m31, mask_mw = mask_sats
    
    # Trace subhalos and add the M31 and the MW Subhalo objects to the
    # ´sim_data´ dictionary, as well as their satellites:
    sub_dict = simtrace_redo.trace_subhalos(
        sim, snap_start, snap_stop
    )
    
    m31 = sub_dict[snap_z0][
        sim.get_snapshot(snap_z0).index_of_halo(m31_id[0], m31_id[1])
    ]
    mw = sub_dict[snap_z0][
        sim.get_snapshot(snap_z0).index_of_halo(mw_id[0], mw_id[1])
    ]

    m31_sats = sub_dict[snap_z0][mask_m31]
    sim_data["M31_Satellites"] = {
        "Snap_id": [np.array([sid for _, sid zip(*sat.get_indices())])
                     for sat in m31_sats],
        "Mass": [np.array([mass[sid][idx] for idx, sid 
                           in zip(*sat.get_indices())])
                 for sat in m31_sats],
        "Vmax": [np.array([vmax[sid][idx] for idx, sid 
                           in zip(*sat.get_indices())])
                 for sat in m31_sats],
        "Radius": []
        "COP": [np.array([cop[sid][idx] for idx, sid 
                           in zip(*sat.get_indices())])
                 for sat in m31_sats]
    }
        
    m31_cop = sub_from_data(m31, cop)
    m31_sat_cops = [sub_from_data(sat, cop) for sat in m31_sats]
    r_m31_sat = [distance(sim.get_snapshot(sid), m31_cop, sat_cop)
                 for sid
        
        np.linalg.norm(
        dataset_comp.periodic_wrap(sim.get_snapshot(sid), 
                                   m31_cop, sat_cop) \
        - m31_cop
    ) for sid, m31_cop, sat_cop in zip()]

In [ ]:
def distance(snap, r1, r2):
    # Compute periodic wrap for r2 around r1:
    r2 = dataset_comp.periodic_wrap(snap, r1, r2)
    
    return np.linalg.norm(r1 - r2)

In [ ]:
print(m31_sat_mass)

In [ ]:
    
    m31_satellites = {
        "Mass": np.array([sat.get_halo_data('MassType', snap_ids) 
                          for sat in subdict[snap_z0][mask_m31]]),
        "Vmax": np.array([sat.get_halo_data("Max_Vcirc", h5_group="Extended", 
                                            snap_ids) 
                          for sat in subdict[snap_z0][mask_m31]]),
    }
    
    sim_data.update({
        "M31": m31_satellites
    })

To SPEED things up a bit, perhaps rather use these:

In [ ]:
def get_data_in_snapshots(simul, snap_ids, dset_name, h5_group='Subhalo'):
    data = {snap_id: simul.snapshots[snap_id].get_subhalos(
        dset_name, h5_group=h5_group
    ) for snap_id in snap_ids}

    return data

In [ ]:
def get_subhalo_data(subhalo, data):
    inds, snap_ids = subhalo.get_indices()
    subdata = np.array([
        data[sid][i] for i, sid in zip(inds, snap_ids)
    ])
    return subdata

In [ ]:
    sim_data.update({
        "M31": {
            "Subhalo": sub_dict[snap_z0][
                sim.get_snapshot(snap_z0).index_of_halo(m31_id[0], m31_id[1])
            ],
            "Satellites": sub_dict[snap_z0][mask_m31]
        },
        "MW": {
            "Subhalo": sub_dict[snap_z0][
                sim.get_snapshot(snap_z0).index_of_halo(mw_id[0], mw_id[1])
            ],
            "Satellites": sub_dict[snap_z0][mask_m31]
        }
    }) 

---

## Retrieve and Compute Datasets for Plotting

Below we add to the data dictionaries of each simulation,,
- the masses (of different types) of the centrals
- their relative distance
- the Hubble expansion of that distance
- the radial and tangential components of their relative peculiar velocity
- and the corresponding redshifts and lookback times